In [ ]:
import io, os, ssl, ast, json, http, socket, queue, struct, sqlite3, threading, logging, datetime 
import flask, flask_basicauth, flask_cors  
#from IPython.display import clear_output

server_ip    = 'guoxiaokang.net'
img_port     = 54321
cmd_port     = 54322
flask_port   = 54323
sensor_port  = 54324

with open('src/map.html', 'r') as fr: mapPage = fr.read()
with open('src/rainbow.jpg', 'rb') as fr: rainbow = fr.read() 
with open('secret/basicAuth.conf', 'r') as fr: basicAuth = fr.read()
basic_auth_username = basicAuth.split('\n')[0]
basic_auth_password = basicAuth.split('\n')[1]
server_cert_letsencrypt  = f'{os.environ["HOME"]}/openssl/fullchain.pem'  # secret of my apache and flask site
server_key_letsencrypt   = f'{os.environ["HOME"]}/openssl/privkey.pem'    # secret of my apache and flask site
server_cert_self_signed  = 'secret/server.crt'    # shared with both rover and driver for identity authentication of server 
server_key_self_signed   = 'secret/server.key'    
client_certs_self_signed = 'secret/clients.crt'   # for identity authentication of rover and driver

ssl_context_self_signed = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH, cafile=client_certs_self_signed)  
ssl_context_self_signed.verify_mode = ssl.CERT_REQUIRED
ssl_context_self_signed.load_cert_chain(certfile=server_cert_self_signed, keyfile=server_key_self_signed) 
cmd_queue = queue.Queue()    # cache the command to be forwarded  

## Control Link

In [ ]:
def getCommonName(ss):
    for i in ss.getpeercert()['subject']:
        if i[0][0]=='commonName':
            return i[0][1] 

class XboxController():
    def __init__(self): 
        self.maneuver_limit = 0.3 
        self.ABS_X          = 0
        self.ABS_Y          = 0
        self.gimbal_coeff   = 0.001 
        self.rot_max        = 2500
        self.rot_front      = 2200 
        self.rot_rear       = 1000 # back 1000 ~ front 2200
        self.rot_min        = 600
        self.ele_max        = 2200
        self.ele_horiz      = 1850 
        self.ele_min        = 1500
        self.rot            = self.rot_front
        self.ele            = self.ele_horiz
    def maneuver(self):
        L = abs(self.ABS_X) * self.ABS_X - abs(self.ABS_Y) * self.ABS_Y
        R = -1*abs(self.ABS_X) * self.ABS_X - abs(self.ABS_Y) * self.ABS_Y  
        if   L >  1 :  L =  1
        elif L < -1 :  L = -1
        elif L > -1 * self.maneuver_limit and L < self.maneuver_limit: L = 0
        if   R >  1 :  R =  1
        elif R < -1 :  R = -1
        elif R > -1 * self.maneuver_limit and R < self.maneuver_limit: R = 0
        #kit.motor3.throttle = L
        #kit.motor4.throttle = R 
        cmd_queue.put(struct.pack('<hf', 1, L))
        cmd_queue.put(struct.pack('<hf', 2, R)) 
    def action(self, code, value):    # 31000 32000  1111 
        if code == -1: 
            cmd_queue.put(struct.pack('<hf', -1, 0))
        elif code == 0:
            self.ABS_X = (value-32767)/32768        # 65535/2
            self.maneuver()
        elif code == 1: 
            self.ABS_Y = (value-32767)/32768        # 65535/2 
            self.maneuver() 
        elif code == 2 and (value > 34000 or value < 31000):   # 2    ->  ABS_Z      ->  rot  
            self.rot -= (value - 32767)*self.gimbal_coeff 
            if   self.rot > self.rot_max: self.rot = self.rot_max
            elif self.rot < self.rot_min: self.rot = self.rot_min
            cmd_queue.put(struct.pack('<hf', 3, self.rot)) 
        elif code == 5 and (value > 35000 or value < 33000):   # 5    ->  ABS_RZ     ->  ele 
            self.ele -= (value - 32767)*self.gimbal_coeff 
            if   self.ele > self.ele_max: self.ele = self.ele_max
            elif self.ele < self.ele_min : self.ele = self.ele_min
            cmd_queue.put(struct.pack('<hf', 4, self.ele))
        elif code == 304:
            cmd_queue.put(struct.pack('<hf', 3, self.rot_front)) 
            cmd_queue.put(struct.pack('<hf', 4, self.ele_horiz))  
            
xbox = XboxController() 

In [ ]:
def cmd_recvd(ss): 
    while True:
        try:
            data = ss.recv(8) 
            code, value = struct.unpack('<hi', data)  
            #print('in', code, value)
            xbox.action(code, value) 
        except socket.timeout:  
            ss.shutdown(socket.SHUT_RDWR)  
            ss.close()
            print('driver timeout')
            break
        except ConnectionResetError: 
            print('driver ConnectionResetError')
            break
        
def cmd_sendd(ss): 
    while True: 
        data = cmd_queue.get()
        try:
            #print('data', *struct.unpack('<hf', data))
            ss.send(data) # Send code and value to server 
        except socket.timeout:
            ss.shutdown(socket.SHUT_RDWR)  
            ss.close() 
            print('rover timeout')
            break
        except BrokenPipeError: 
            print('rover BrokenPipeError')
            break
        except ConnectionResetError: 
            print('rover ConnectionResetError')
            break

In [ ]:
def control_link_daemon(): 
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    s.bind((server_ip, cmd_port))
    s.listen(0)
    while True: 
        print(f"Waiting for incoming connection")
        c, a = s.accept() 
        ss = ssl_context_self_signed.wrap_socket(c, server_side=True)
        ss.settimeout(30)
        name = getCommonName(ss)
        print(f"Connection with {name} established ~")
        if name == 'driver': 
            threading.Thread(target=cmd_recvd, args=(ss,)).start()
        elif name == 'rover':
            threading.Thread(target=cmd_sendd, args=(ss,)).start() 
        
threading.Thread(target=control_link_daemon).start()

In [ ]:
a = b',,,,,,,32,359.94,-3.50,0.00,4.57\r\n' 

lat, lon, alt, course, speed, sat, timestamp, temp, x, y, z, vol = a.strip().decode().split(',')

In [ ]:
raise

In [ ]:
class SENSOR(): # sensor logging (plus GPS logging/insertion)
    def __init__(self):
        self.data = {'T':0, 'X':0, 'Y':0, 'Z':0, 'focus':'true'}
        self.long  = struct.calcsize('<H')
        threading.Thread(target=self.updaterd).start() 

    def updaterd(self):
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((server_ip, sensor_port))
        s.listen(0)
        while True: 
            c, a = s.accept() 
            ss = ssl_context_self_signed.wrap_socket(c, server_side=True).makefile('rb')
            while True:
                try:
                    data_len, = struct.unpack('<H', ss.read(self.long)) 
                    print(ss.read(data_len)) 
                except socket.timeout:  
                    ss.shutdown(socket.SHUT_RDWR)  
                    ss.close()
                    print('driver timeout')
                    break
                except ConnectionResetError: 
                    print('driver ConnectionResetError')
                    break
sensor = SENSOR()

class MJPEG():# image cache
    def __init__(self):
        self.frame = rainbow # default image
        self.long  = struct.calcsize('<L')  
        threading.Thread(target=self.imgd).start() 

    def imgd(self): 
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((server_ip, img_port))
        s.listen(0)
        while True: 
            c, a = s.accept() 
            ss = ssl_context_self_signed.wrap_socket(c, server_side=True).makefile('rb')      
            while True:
                try:
                    img_len, = struct.unpack('<L', ss.read(self.long)) 
                    self.frame = ss.read(img_len)  
                except socket.timeout:  
                    ss.shutdown(socket.SHUT_RDWR)  
                    ss.close()
                    print('driver timeout')
                    break
                except ConnectionResetError: 
                    print('driver ConnectionResetError')
                    break         
mjpeg = MJPEG()

#### A class taking care of database operation
the newcomer need to retrieve all GPS tracking data and that's the reason of involving a database 

In [ ]:
q_in, q_out = queue.Queue(), queue.Queue()  
def dbd(): # avoid sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread.
    #db = sqlite3.connect(":memory:")
    try:
        os.remove("GPS.db")
    except:
        pass
    db = sqlite3.connect("GPS.db")
    cur = db.cursor() 
    cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", ('tracking',))
    if not cur.fetchone(): cur.execute("CREATE TABLE tracking (lat, lon, timestamp)") 
    while True:
        item = q_in.get()
        if isinstance(item, tuple):
            cur.execute("INSERT INTO tracking (lat, lon, timestamp) VALUES (?, ?, ?)", item) 
        else: 
            if item == 'newcomer':
                cur.execute("SELECT lon, lat, timestamp FROM tracking WHERE timestamp=(SELECT MIN(timestamp) FROM tracking)") 
            else:
                cur.execute("SELECT lon, lat, timestamp FROM tracking WHERE timestamp>=?", (float(item),))
            q_out.put(cur.fetchall())
        q_in.task_done()
        
threading.Thread(target=dbd).start() 

In [ ]:
   
                    if 'P' in j: 
                        GPRMC = pynmea2.parse(j['P']) # http://aprs.gids.nl/nmea/
                        lon, lat = GPRMC.lon, GPRMC.lat
                        lat = float(lat[:2]) + float(lat[2:])/60
                        lon = float(lon[:3]) + float(lon[3:])/60
                        if GPRMC.lat_dir == 'S': lat *= -1 
                        if GPRMC.lon_dir == 'W': lon *= -1
                        timestamp = datetime.datetime.combine(GPRMC.datestamp, GPRMC.timestamp).timestamp() 
                        q_in.join()
                        q_in.put((lat, lon, timestamp)) 
                    if 'N' in j and 'a' in j:
                        self.data['N'] = 180 - int(j['N']) 
                        x, y, z = ast.literal_eval(j['a']) 
                        self.data['x'], self.data['y'], self.data['z'] = x/-20.48, y/20.48, z 

In [ ]:
app = flask.Flask(__name__)
flask_cors.CORS(app) 
app.config['BASIC_AUTH_USERNAME'] = basic_auth_username
app.config['BASIC_AUTH_PASSWORD'] = basic_auth_password
basic_auth = flask_basicauth.BasicAuth(app)

#log = logging.getLogger('werkzeug')
#log.setLevel(logging.ERROR) 

@app.route("/")
@basic_auth.required
def home(): 
    return flask.render_template_string(mapPage, 
        ip_port       = f'{server_ip}:{flask_port}', 
        username      =  basic_auth_username, 
        password      =  basic_auth_password)  

@app.route("/stream.mjpg")
@basic_auth.required
def mjpg():    
    def generator(): 
        while True:  
            frame = mjpeg.frame 
            yield f'''--FRAME\r\nContent-Type: image/jpeg\r\nContent-Length: {len(frame)}\r\n\r\n'''.encode() 
            yield frame
    r = flask.Response(response=generator(), status=200)
    r.headers.extend({'Age':0, 'Content-Type':'multipart/x-mixed-replace; boundary=FRAME',
                      'Pragma':'no-cache', 'Cache-Control':'no-cache, private',}) 
    return r

RESPONSE = sensor.data 
@app.route("/sensor", methods=['GET'])
@basic_auth.required
def sensor():
    global RESPONSE   
    newest_timestamp_client_side = flask.request.args.get('timestamp') 
    q_in.join()  
    q_in.put(newest_timestamp_client_side)
    q_in.join() 
    llts = q_out.get() # lat, long, timestamp 
    if llts:
        RESPONSE.update({
            "rover": {
                "type": "Feature", 
                "geometry": {
                    "type": "Point",
                    "coordinates": llts[-1][:2] # long lat in GeoJson
                }, 
                "properties": {
                    "newestViewCenter": llts[-1][1::-1], # lat long in Google and Openstreetmap
                    "newestTimestampServerSide": llts[-1][2],
                    "homePoint": "false"
                },
            }
        })
        if newest_timestamp_client_side == 'newcomer': 
            RESPONSE['rover']['properties']['homePoint'] = llts[-1][1::-1]
    if len(llts) > 1: 
        RESPONSE.update({
            "trail": {
                "type": "Feature",
                "geometry": {
                    "type": "LineString",
                    "coordinates": [i[:2] for i in llts]
                }
            }
        })
    return flask.jsonify(RESPONSE)

@app.route('/axios.min.js', methods=['GET'])
def axios_js():
        return flask.send_file('src/axios.min.js', mimetype='application/javascript')
    
@app.route('/leaflet.rotatedMarker.js', methods=['GET'])
def rotatedMarker_js():
        return flask.send_file('src/leaflet.rotatedMarker.js', mimetype='application/javascript')
    
@app.route('/home.png', methods=['GET'])
def home_svg():
        return flask.send_file('src/home.png', mimetype='image/png')
    
@app.route('/arrow.png', methods=['GET'])
def arrow_svg():
        return flask.send_file('src/arrow.png', mimetype='image/png')

def flask_in_bg():
    app.run(server_ip, flask_port, ssl_context=(server_cert_letsencrypt, server_key_letsencrypt))
threading.Thread(target=flask_in_bg).start() 
#flask_in_bg()

In [ ]:
class XboxController():
    def __init__(self):
        self.rotation_rear     = 1000 # back 1000 ~ front 2200
        self.rotation_front    = 2200 
        self.elevation_horiz   = 1850 
        self.gimbal_sensitivity= 0.01
        self.rot = self.rotation_front
        self.ele = self.elevation_horiz 
    def action(self, code, value):   
        if code == 0 and (value > 37000 or value < 34000):     # 0    ->  ABS_X      ->  trn 
            increment = (value - 33000)*self.turn_sensitivity
            self.top += increment
            if   self.top > self.turn_right_top: self.top = self.turn_right_top
            elif self.top < self.turn_left_top : self.top = self.turn_left_top 
            self.bot -= increment
            if   self.bot > self.turn_left_bottom : self.bot = self.turn_left_bottom
            elif self.bot < self.turn_right_bottom: self.bot = self.turn_right_bottom   
            ss_rover.send(struct.pack('<Hi', code, int(self.top)))  
        elif code == 2 and (value > 34000 or value < 31000):   # 2    ->  ABS_Z      ->  rot  
            self.rot -= (value - 33000)*self.gimbal_sensitivity
            if   self.rot > self.rotation_rear : self.rot = self.rotation_rear 
            elif self.rot < self.rotation_front: self.rot = self.rotation_front
            ss_rover.send(struct.pack('<Hi', code, int(self.rot)))
        elif code == 5 and (value > 35000 or value < 33000):   # 5    ->  ABS_RZ     ->  ele     
            self.ele += (value - 33000)*self.gimbal_sensitivity 
            if   self.ele > self.elevation_horiz: self.ele = self.elevation_horiz
            elif self.ele < self.elevation_sky  : self.ele = self.elevation_sky 
            ss_rover.send(struct.pack('<Hi', code, int(self.ele)))
        elif code ==  10:
            #if value 
            ss_rover.send(struct.pack('<Hi', code, int(value/1023*255)))     # 10   ->  ABS_BRAKE  ->  rev  
        elif code ==   9: 
            ss_rover.send(struct.pack('<Hi', code, int(value/1023*255)))     # 9    ->  ABS_GAS    ->  drv   
        elif code ==  17: # sensitivity
            enum = [0.00001, 0.0001, 0.001, 0.01]
            idx  = enum.index(self.gimbal_sensitivity) 
            idx -= value 
            if idx >= 0 and idx < len(enum): 
                self.gimbal_sensitivity = enum[idx] 
        elif code == 304: ss_rover.send(struct.pack('<Hi', code, 1350))     # 304  ->  BTN_A      ->  rst
        elif code == 305: 
            global Sensor
            if Sensor['focus'] == 'true': 
                Sensor['focus'] = 'false'
            else:
                Sensor['focus'] = 'true'
xbox = XboxController() 